In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler 
import os
import re

In [2]:
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

In [3]:
test_ids = test.index

In [4]:
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dates'].dt.date)
    data['n_days'] = (data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    data["XY"] = data["X"] - data["Y"]
    data.drop(columns=['Dates','Date','Address'], inplace=True)
    
    return data

In [5]:
train = feature_engineering(train) 
test = feature_engineering(test)

In [6]:
y_dummies = pd.get_dummies(train.Category).values

In [7]:
y_dummies

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
le2 = LabelEncoder()
y= le2.fit_transform(train['Category'])

train.drop(columns=['Descript','Resolution', "Category"], inplace=True)

In [9]:
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

train.Block = train.Block.apply(lambda x:1 if x == True else 0)
test.Block = test.Block.apply(lambda x:1 if x == True else 0)

In [10]:
train.head()

,DayOfWeek,PdDistrict,X,Y,n_days,Day,Month,Year,Hour,Minute,Block,XY
0,2,4,-122.425892,37.774599,4510,13,5,2015,23,53,0,-160.200490
1,2,4,-122.425892,37.774599,4510,13,5,2015,23,53,0,-160.200490
2,2,4,-122.424363,37.800414,4510,13,5,2015,23,33,0,-160.224777
3,2,4,-122.426995,37.800873,4510,13,5,2015,23,30,1,-160.227868
4,2,5,-122.438738,37.771541,4510,13,5,2015,23,30,1,-160.210279


In [11]:
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [12]:
from keras import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [13]:
model = Sequential()

In [14]:
model.add(Dense(1536, input_dim=train.shape[1], activation = "relu"))

In [15]:
model.add(Dropout(0.5))

In [16]:
model.add(Dense(1280, activation = "relu"))
model.add(Dropout(0.5))

In [17]:
model.add(Dense(39, activation = "softmax"))

In [18]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
model.fit(train, y_dummies, batch_size=4096, epochs=120, verbose=1)
predictions = model.predict(test, batch_size = 8192, verbose=1)
submission = pd.DataFrame(predictions, columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')), index=test_ids)
submission.to_csv('NN.csv', index_label='Id') 

Epoch 1/120
878049/878049 [==============================] - 10s 11us/step - loss: 2.5704 - acc: 0.2305
Epoch 2/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4874 - acc: 0.2506
Epoch 3/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4636 - acc: 0.2558
Epoch 4/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4497 - acc: 0.2594
Epoch 5/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4389 - acc: 0.2624
Epoch 6/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4283 - acc: 0.2653
Epoch 7/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4195 - acc: 0.2678
Epoch 8/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4127 - acc: 0.2697
Epoch 9/120
878049/878049 [==============================] - 8s 9us/step - loss: 2.4070 - acc: 0.2714
Epoch 10/120
878049/878049 [==============================] - 8s 9us/step - loss